<h2><span style  = "background-color: lightblue;">Red Brand Canners - LP way of solving Production Conundrum</span></h3>

In [ ]:
!pip install gurobipy

import gurobipy as gp

from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np

<b>Objective Function Formulation:</b>
The objective function is designed to maximize the profit contribution of RBC's products, considering the utilization of grade A and B tomatoes across various product lines. By excluding the sunk costs associated with the procurement of grade A and B tomatoes from the variable costs, we derived the contribution per product line and calculated the profit contribution per 1000 pounds for Whole Tomatoes, Tomato Juice, and Tomato Paste. The objective function is formulated as follows:

Maximize 
$$246.67 (a_w + b_w) + 198 (a_j + b_j) + 222 (a_p + b_p)$$

**Variables:**

$$
\begin{aligned}
a_w &= \text{weight of A grade tomatoes used in the production of Whole Tomatoes} \\
b_w &= \text{weight of B grade tomatoes used in the production of Whole Tomatoes} \\
a_j &= \text{weight of A grade tomatoes used in the production of Tomato Juice} \\
b_j &= \text{weight of B grade tomatoes used in the production of Tomato Juice} \\
a_p &= \text{weight of A grade tomatoes used in the production of Tomato Paste} \\
b_p &= \text{weight of B grade tomatoes used in the production of Tomato Paste}
\end{aligned}
$$

<b>Constraints:</b> 
The constraints are categorized into Demand, Supply, and Quality considerations. The demand constraints ensure that the total weight of grade A and B tomatoes used does not exceed the required quantities for each product. Supply constraints reflect the limited availability of grade A and B tomatoes, while quality constraints guarantee that the products meet RBC's specified quality standards.

**Mathematical Formulation of Constraints**

**Demand Constraints:**
$$
\begin{aligned}
a_w + b_w &\leq 14400 \\
a_j + b_j &\leq 1000 \\
a_p + b_p &\leq 2000
\end{aligned}
$$

**Supply Constraints:**
$$
\begin{aligned}
a_w + a_j + a_p &\leq 600 \\
b_w + b_j + b_p &\leq 2400
\end{aligned}
$$

**Quality Constraints:**
$$
\begin{aligned}
9a_w + 5b_w &\geq 8(a_w + b_w) \\
9a_j + 5b_j &\geq 8(a_j + b_j)
\end{aligned}
$$

## 1. Without the extra 80,000

In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw)+198*(aj+bj)+222*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+aj+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [14400, 1000, 2000, 600, 2400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x0d2efda0
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4000000e+05   9.624497e+02   0.000000e+00      0s
       2    6.7606667e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.760666667e+05

Profit contribution: 676067

Decision 

## 2. With the Extra 80,000 of grade A
#### Additional supply of A tomatoes
If RBC procures an additional 80,000 pounds of grade A tomatoes at 25.5 cents per pound, they must utilize them in production to maximize their profit. To achieve this, we need to formulate a new objective function and adjust the cost of procurement. The procurement cost should be distributed across the product lines and then subtracted from the objective function, as this cost would otherwise reduce the profit.
The cost of procurement for each of the three product lines is $255 per 1000 pounds. For Whole Tomatoes, this cost is calculated as 25.5 cents/pound × 18 pounds/case/100/18 pounds/case. The same formula is applied to calculate the cost for the other two product lines.

<i>The new objective function is as follows:</i>


**Objective Function:**

Maximize
$$
246.67(a_w + b_w) + 198(a_j + b_j) + 222(a_p + b_p) - 255(a_{aw} + a_{aj} + a_{ap})
$$

**New Variables:**

$$
\begin{aligned}
a_{aw} &= \text{weight of A grade additional tomatoes used in the production of Whole Tomatoes} \\
a_{aj} &= \text{weight of A grade additional tomatoes used in the production of Tomato Juice} \\
a_{ap} &= \text{weight of A grade additional tomatoes used in the production of Tomato Paste}
\end{aligned}
$$

In [6]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")
aaw = m.addVar(name="aaw")
aaj = m.addVar(name="aaj")
aap = m.addVar(name="aap")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw+aaw)+198*(aj+bj+aaj)+222*(ap+bp+aap)-255*(aaw+aaj+aap)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw+aaw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj+aaj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp+aap<=2000, name='p_dem')


# Supply constraints
con4 = m.addConstr(aw+aj+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2400, name='b_sup')
conxa = m.addConstr(aaw+aaj+aap<=80, name='aa_supply')

# Quality constraints
con6 = m.addConstr(9*aw+9*aaw+5*bw>=8*(aw+aaw+bw), name='w_qual')
con7 = m.addConstr(9*aj+9*aaj+5*bj>=6*(aj+aaj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp','aaw','aaj','aap']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x,aaw.x,aaj.x,aap.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC, aaw.RC, aaj.RC, aap.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222, (4.44/18*1000) - 255, 198 - 255, 222 - 255]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp,aaw.SAObjUp,aaj.SAObjUp,aap.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow, aaw.SAObjLow, aaj.SAObjLow, aap.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual', 'aa_supply']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi, conxa.pi]),
    ('RHS Coeff', [14400, 1000, 2000, 600, 2400, 0, 0, 80]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack, conxa.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp, conxa.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow, conxa.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x7ccba7f6
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [8e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4529491e+05   1.660224e+02   0.000000e+00      0s
       3    6.7734667e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.773466667e+05

Profit contribution: 677347

Decision variables:
aw = 535
aj = 65
ap = 0
bw = 205
bj = 195
bp = 2000
aaw = 

## 3. Advertising 


Through marketing campaigns, there is a potential to increase the demand for certain product categories, which can lead to increased profitability. If RBC considers creating a surplus demand of <b>5000</b> cases, they should focus on Tomato Paste. The Sensitivity Analysis report shows that the Demand constraint for Tomato Paste has a Slack of <b>0</b>, indicating that it is a binding constraint. Therefore, any additional demand up to <b>2173.33 pounds</b> could still result in surplus profit. However, for the other two product categories, increasing demand may not result in additional profit, as they have a non-zero slack (13580 for whole tomato demand and 740 for tomato juice).

We revised the demand constraint for Tomato Paste from <b>2000 to 2125</b> (calculated as <b>25 lb/case * (80,000 + 5000))</b>, scaling the number by dividing it by <b>1000</b> as done in previous LP models, and executed a new LP model. As expected, we observed a profit increase of <b>$6041 (=$683,388 – $677,347)</b>. This change in profit can be validated by multiplying the shadow price of <b>48.33 by 125 (2125 – 2000)</b>.


In [8]:
125*48.333333

6041.666625000001

Based on the shaow price 

In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")
aaw = m.addVar(name="aaw")
aaj = m.addVar(name="aaj")
aap = m.addVar(name="aap")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw+aaw)+198*(aj+bj+aaj)+222*(ap+bp+aap)-255*(aaw+aaj+aap)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw+aaw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj+aaj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp+aap<=2125, name='p_dem')


# Supply constraints
con4 = m.addConstr(aw+aj+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2400, name='b_sup')
conxa = m.addConstr(aaw+aaj+aap<=80, name='aa_supply')

# Quality constraints
con6 = m.addConstr(9*aw+9*aaw+5*bw>=8*(aw+aaw+bw), name='w_qual')
con7 = m.addConstr(9*aj+9*aaj+5*bj>=6*(aj+aaj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp','aaw','aaj','aap']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x,aaw.x,aaj.x,aap.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC, aaw.RC, aaj.RC, aap.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, 222, (4.44/18*1000), 198, 222, (4.44/18*1000) - 255, 198 - 255, 222 - 255]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp,aaw.SAObjUp,aaj.SAObjUp,aap.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow, aaw.SAObjLow, aaj.SAObjLow, aap.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual', 'aa_supply']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi, conxa.pi]),
    ('RHS Coeff', [14400, 1000, 2125, 600, 2400, 0, 0, 80]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack, conxa.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp, conxa.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow, conxa.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8 rows, 9 columns and 24 nonzeros
Model fingerprint: 0xd48e3f77
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [8e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+04]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7304491e+05   1.816474e+02   0.000000e+00      0s
       3    6.8338833e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.833883333e+05

Profit contribution: 683388

Decision variables:
aw = 581.875
aj = 18.125
ap = 0
bw = 220.625
bj = 54.375
b

## 4. Additional supply of B tomatoes

Yes, because when at the <b>18 cents per pound</b> cost the <b>shadow price is 173.666</b> which shows an increase in profit when we add one unit to the supply. 

## 5. Closing down production lines
Starting up the production line for each product incurs a significant set-up cost, which plays a pivotal role in production planning and budgeting by directly influencing the feasibility and profitability of manufacturing. When considering the termination of a product category, it is essential to incorporate the cost savings into the objective function. This approach allows for a thorough evaluation of profitability and aids in making informed decisions regarding which product line to close, aiming to maximize profits for RBC management.




In [14]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
##ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
##bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw)+198*(aj+bj)+50
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
##con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+aj<=600, name='a_sup')
con5 = m.addConstr(bw+bj<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'bw', 'bj']),
    ('Final Value', [aw.x, aj.x, bw.x, bj.x,]),
    ('Reduced Cost', [aw.RC, aj.RC, bw.RC, bj.RC]),
    ('Obj Coeff', [(4.44/18*1000), 198, (4.44/18*1000), 198]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, bw.SAObjUp, bj.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, bw.SAObjLow, bj.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [14400, 1000, 600, 2400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0xaa5b4930
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1316111e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.131611111e+05

Profit contribution: 313161

Decision variables:
aw = 350
aj = 250
bw = 116.667
bj = 750


  Name  Final Value  Reduced Cost   Obj Coeff  Upper Range   Lower Range
0   aw   35

In [15]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
##aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
##bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 50+198*(aj+bj)+222*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
##con1 = m.addConstr(aw+bw<=14400, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aj+ap<=600, name='a_sup')
con5 = m.addConstr(bj+bp<=2400, name='b_sup')

# Quality constraints
##con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', [ 'aj', 'ap', 'bj', 'bp']),
    ('Final Value', [ aj.x, ap.x, bj.x, bp.x]),
    ('Reduced Cost', [ aj.RC, ap.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [198, 222, 198, 222]),
    ('Upper Range', [ aj.SAObjUp, ap.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [ aj.SAObjLow, ap.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', [ 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'j_qual']),
    ('Shadow Price', [ con2.Pi, con3.Pi, con4.Pi, con5.Pi,  con7.Pi]),
    ('RHS Coeff', [ 1000, 2000, 600, 2400, 0]),
    ('Slack', [con2.Slack, con3.Slack, con4.Slack, con5.Slack, con7.Slack]),
    ('Upper Range', [ con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp,  con7.SARHSUp]),
    ('Lower Range',
     [ con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0xbc73888d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 2e+03]
Presolve time: 0.00s
Presolved: 5 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4000000e+32   8.000000e+30   8.400000e+02      0s
       4    6.4205000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.420500000e+05

Profit contribution: 642050

Decision variables:
aj = 250
ap = 350
bj = 750
bp = 1650


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range

In [16]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomato usage
aw = m.addVar(name="aw")
##aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
##bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = (4.44/18*1000)*(aw+bw)+222*(ap+bp)+50
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=14400, name='w_dem')
##con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply constraints
con4 = m.addConstr(aw+ap<=600, name='a_sup')
con5 = m.addConstr(bw+bp<=2400, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
##con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'ap', 'bw', 'bp']),
    ('Final Value', [aw.x, ap.x, bw.x, bp.x]),
    ('Reduced Cost', [aw.RC, ap.RC, bw.RC, bp.RC]),
    ('Obj Coeff', [(4.44/18*1000), 222, (4.44/18*1000), 222]),
    ('Upper Range', [aw.SAObjUp, ap.SAObjUp, bw.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, ap.SAObjLow, bw.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem',  'p_dem', 'a_sup', 'b_sup', 'w_qual']),
    ('Shadow Price', [con1.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi]),
    ('RHS Coeff', [14400, 2000, 600, 2400, 0]),
    ('Slack', [con1.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack]),
    ('Upper Range', [con1.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 4 columns and 10 nonzeros
Model fingerprint: 0x071cbe57
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4138333e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.413833333e+05

Profit contribution: 641383

Decision variables:
aw = 600
ap = 0
bw = 200
bp = 2000


  Name  Final Value  Reduced Cost   Obj Coeff  Upper Range   Lower Range
0   aw        60

## 6. RBC Tomato Purchase Strategy Analysis a year later
We have revised the profit objective formula as it does include the <b>purchasing price</b>:


**Objective Function:**

Maximize
$$
47(a_w + b_w) - 2(a_j + b_j) + 22(a_p + b_p)
$$

### For only Sunny

In [19]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=10000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_allsup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_allsup')

# Supply Wet  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_Wsup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=600, name='a_Nsup')
##con5 = m.addConstr(bw+bj+bp<=2400, name='b_Nsup')

# Supply Sunny 
con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [10000, 1000, 2000, 7800 , 5200, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x78ee7739
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1400000e+05   2.750000e+02   0.000000e+00      0s
       1    5.1400000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.140000000e+05

Profit contribution: 514000

Decision variables:
aw = 7800
aj = 0
ap = 0
bw = 2200
bj = 0
bp = 2000


  Name

### For Normal

In [21]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=10000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Wet  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_Wsup')

# Supply Normal
con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [10000, 1000, 2000, 6500 , 6500, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x1c19962c
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5133333e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.513333333e+05

Profit contribution: 451333

Decision variables:
aw = 6500
aj = 0
ap = 0
bw = 2166.67
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range

### For Bad and Wet 

In [23]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=10000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Wet  
con4 = m.addConstr(aw+aj+ap<=2600, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=10400, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [10000, 1000, 2000, 2600 , 10400, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0xc81e4eea
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0693333e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.069333333e+05

Profit contribution: 206933

Decision variables:
aw = 2600
aj = 0
ap = 0
bw = 866.667
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range

Sunny pounds of tomatoes to buy : 12,000,000


Wet pounds of tomatoes to buy : 5,467,000

Normal pounds of tomatoes to buy : 10,667,000

### Part 2
Suppose you order <b>S</b> pounds of tomatoes. What would be the possible outcomes, given that the year could be sunny, normal or poor? Perform a scenario analysis. Do the same for ordering <b>N</b> and <b>P</b> tomatoes.
Now first if the orders are based on <b>S</b> but its a Normal year. 

In [26]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=9000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=6000, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=6000, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [9000, 1000, 2000, 6000 , 6000, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x534933e1
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 9e+03]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.200000000e+05

Profit contribution: 420000

Decision variables:
aw = 6000
aj = 0
ap = 0
bw = 2000
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range
0 

Now if we had the order for sunny but it was a wet year

In [28]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=9000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Wet when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=2400, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=9600, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [9000, 1000, 2000, 2400 , 9600, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x6f459498
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9440000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.944000000e+05

Profit contribution: 194400

Decision variables:
aw = 2400
aj = 0
ap = 0
bw = 800
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range
0  

Now if the orders were based on Normal but the year is sunny 

In [30]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=7667, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=6400.2, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=4266.8, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [7667, 1000, 2000, 6400.2 , 4266.8, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x49f9fe4b
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 8e+03]
Presolve removed 5 rows and 4 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0434900e+05   1.583500e+02   0.000000e+00      0s
       1    4.0434900e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.043490000e+05

Profit contribution: 404349

Decision variables:
aw = 6400.2
aj = 0
ap = 0
bw = 1266.8
bj = 0
bp = 2000


  

Now if the orders are Normal but the year is wet 

In [32]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=7667, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=2133.4, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=8533.6, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [7667, 1000, 2000, 2133.4 , 8533.6, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x3a7d075d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 9e+03]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7769307e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.776930667e+05

Profit contribution: 177693

Decision variables:
aw = 2133.4
aj = 0
ap = 0
bw = 711.133
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Ran

Now if the Orders are based on Wet but the year is Sunny

In [34]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=2467, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=3280.2, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2186.8, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [2467, 1000, 2000, 3280.2 , 2186.8, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0xa33c6fb3
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 3e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 5 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5994900e+05   1.483500e+02   0.000000e+00      0s
       1    1.5994900e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.599490000e+05

Profit contribution: 159949

Decision variables:
aw = 2467
aj = 0
ap = 813.2
bw = 0
bj = 0
bp = 1186.8


  

Now if the orders are based on Wet but the year is Normal

In [69]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=2467, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=2733.5, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=2733.5, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [2467, 1000, 2000, 2733.5 , 2733.5, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x244c470f
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 3e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 5 rows, 4 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5994900e+05   2.166875e+02   0.000000e+00      0s
       1    1.5994900e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.599490000e+05

Profit contribution: 159949

Decision variables:
aw = 2467
aj = 0
ap = 266.5
bw = 0
bj = 0
bp = 1733.5


  

Now what if the orders are based on sunny and the year is also sunny 

In [38]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=9000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=7200, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=4800, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0xa96884ac
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 9e+03]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6700000e+05   2.250000e+02   0.000000e+00      0s
       1    4.6700000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.670000000e+05

Profit contribution: 467000

Decision variables:
aw = 7200
aj = 0
ap = 0
bw = 1800
bj = 0
bp = 2000


Now what if the Orders are based on Wet and the year is also wet 

In [40]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=2467, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
##con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Normal when Order was for sunny  
con4 = m.addConstr(aw+aj+ap<=1093.4, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=4373.6, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x04348e5e
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 4e+03]
Presolve removed 7 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1251973e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.125197333e+05

Profit contribution: 112520

Decision variables:
aw = 1093.4
aj = 0
ap = 0
bw = 364.467
bj = 0
bp = 2000


Now What if the Orders are based on Normal and the year is also Normal 

In [42]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=7667, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
con4 = m.addConstr(aw+aj+ap<=5333.5, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=5333.5, name='b_sup')

# Supply Wet  
##con4 = m.addConstr(aw+aj+ap<=2600, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=10400, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0xba8b1f67
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 8e+03]
Presolve removed 7 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7823267e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.782326667e+05

Profit contribution: 378233

Decision variables:
aw = 5333.5
aj = 0
ap = 0
bw = 1777.83
bj = 0
bp = 2000


In [43]:

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from collections import OrderedDict

# Model
m = gp.Model("RBC")

# Create decision variables for tomatoes usage
aw = m.addVar(name="aw")
aj = m.addVar(name="aj")
ap = m.addVar(name="ap")
bw = m.addVar(name="bw")
bj = m.addVar(name="bj")
bp = m.addVar(name="bp")

# The objective is to maximize the profit contribution
obj = 47*(aw+bw)-2*(aj+bj)+22*(ap+bp)
m.setObjective(obj, GRB.MAXIMIZE)

# Demand constraints
con1 = m.addConstr(aw+bw<=10000, name='w_dem')
con2 = m.addConstr(aj+bj<=1000, name='j_dem')
con3 = m.addConstr(ap+bp<=2000, name='p_dem')

# Supply for all with  all the probabilities included  
con4 = m.addConstr(aw+aj+ap<=5850, name='a_sup')
con5 = m.addConstr(bw+bj+bp<=7150, name='b_sup')

# Supply Wet  
##con4 = m.addConstr(aw+aj+ap<=2600, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=10400, name='b_Wsup')

# Supply Normal
##con4 = m.addConstr(aw+aj+ap<=6500, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=6500, name='b_sup')

# Supply Sunny 
##con4 = m.addConstr(aw+aj+ap<=7800, name='a_sup')
##con5 = m.addConstr(bw+bj+bp<=5200, name='b_sup')

# Quality constraints
con6 = m.addConstr(9*aw+5*bw>=8*(aw+bw), name='w_qual')
con7 = m.addConstr(9*aj+5*bj>=6*(aj+bj), name='j_qual')

# Non-negativity constraints omitted as default in 'addVar'

# Solve
m.optimize()

# Print optimal value of the objective function
print('\nProfit contribution: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

# Create table for decision variables' sensitivity analysis
decision_var = OrderedDict([
    ('Name', ['aw', 'aj', 'ap', 'bw', 'bj', 'bp']),
    ('Final Value', [aw.x, aj.x, ap.x, bw.x, bj.x, bp.x]),
    ('Reduced Cost', [aw.RC, aj.RC, ap.RC, bw.RC, bj.RC, bp.RC]),
    ('Obj Coeff', [47, -2, 22, 47, -2, 22]),
    ('Upper Range', [aw.SAObjUp, aj.SAObjUp, ap.SAObjUp, bw.SAObjUp, bj.SAObjUp, bp.SAObjUp]),
    ('Lower Range', [aw.SAObjLow, aj.SAObjLow, ap.SAObjLow, bw.SAObjLow, bj.SAObjLow, bp.SAObjLow])
])


# Create table for constraints' sensitivity analysis
constraint = OrderedDict([
    ('Name', ['w_dem', 'j_dem', 'p_dem', 'a_sup', 'b_sup', 'w_qual', 'j_qual']),
    ('Shadow Price', [con1.Pi, con2.Pi, con3.Pi, con4.Pi, con5.Pi, con6.Pi, con7.Pi]),
    ('RHS Coeff', [10000, 1000, 2000, 5850 , 7150, 0, 0]),
    ('Slack', [con1.Slack, con2.Slack, con3.Slack, con4.Slack, con5.Slack, con6.Slack, con7.Slack]),
    ('Upper Range', [con1.SARHSUp, con2.SARHSUp, con3.SARHSUp, con4.SARHSUp, con5.SARHSUp, con6.SARHSUp, con7.SARHSUp]),
    ('Lower Range',
     [con1.SARHSLow, con2.SARHSLow, con3.SARHSLow, con4.SARHSLow, con5.SARHSLow, con6.SARHSLow, con7.SARHSLow])
])

# Print sensitivity analysis tables for decision variables and constraints
print('\n')
print(pd.DataFrame.from_dict(decision_var))
print('\n')
print(pd.DataFrame.from_dict(constraint))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 20.04.4 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x4053ea03
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+04]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1060000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.106000000e+05

Profit contribution: 410600

Decision variables:
aw = 5850
aj = 0
ap = 0
bw = 1950
bj = 0
bp = 2000


  Name  Final Value  Reduced Cost  Obj Coeff  Upper Range  Lower Range
0 

Including all probabilites the total pound of tomato that should be purchese is 9,800,000